# Load video and model
## Video:
- UTDD: BVK019, BVK022, BVK024, BVK029, BVK042, BVK066

- UTTQ: CS101, CS201, BVK037, BVK040, BVK083, BVK091

- test_6s

- test_3s

## Model
- thucquan.pt: ['2_Viem_thuc_quan', '5_Ung_thu_thuc_quan']

- daday.pt: ['3_Viem_da_day_HP_am', '4_Viem_da_day_HP_duong', '6_Ung_thu_da_day']

- htt.pt: ['7_Loet_HTT']

- best0903.pt, best2602.pt: ['polyp', 'esophagael cancer']


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/ENDOCV/video_test/UTTQ /content/drive/MyDrive/ENDOCV/video_test/UTDD /content

In [ ]:
#!cp /content/drive/MyDrive/ENDOCV/video_test/UTTQ/BVK037.mp4 /content/drive/MyDrive/ENDOCV/video_test/UTTQ/CS101.mp4 /content

In [ ]:
#!cp /content/drive/MyDrive/ENDOCV/video_test/UTDD/BVK042.mp4 /content/drive/MyDrive/ENDOCV/video_test/UTTQ/CS201.mp4 /content/drive/MyDrive/ENDOCV/video_test/UTTQ/BVK037.mp4 /content

In [ ]:
!cp /content/drive/MyDrive/ENDOCV/model_pt/model_yolo/thucquan.pt /content/drive/MyDrive/ENDOCV/model_pt/model_yolo/daday.pt /content

In [ ]:
!cp /content/drive/MyDrive/ENDOCV/model_pt/best0903.pt /content/drive/MyDrive/ENDOCV/model_pt/best2602.pt /content

# Install requirements

In [ ]:
!pip install torchvision==0.17.1+cu121 -f https://download.pytorch.org/whl/torch_stable.html
!pip install ultralytics
!pip install boxmot==10.0.51

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# !git clone https://github.com/mikel-brostrom/yolo_tracking.git
# !pip install -v -e .

In [ ]:
# !git clone https://github.com/KeeganFernandesWork/yolo_tracking
# %cd yolo_tracking
# !pip install -r requirements.txt
# !pip install .

# Track

In [ ]:
from ultralytics import YOLO
from pathlib import Path
from time import perf_counter
import cv2
import numpy as np
import torch
from boxmot import (OCSORT, BoTSORT, BYTETracker, DeepOCSORT, StrongSORT, create_tracker, get_tracker_config)
from pathlib import Path
import sys
import datetime
import pandas as pd
from google.colab.patches import cv2_imshow

In [ ]:
test_vid = "/content/UTTQ/CS101.mp4"
model_weights = "/content/thucquan.pt"

input_video_name = test_vid.split("/")[-1].split(".")[0] + '_' + test_vid.split("/")[-2].split(".")[0]
s
# Tạo từ điển ánh xạ giữa tên model_weights và model_classes
model_classes_dict = {
    "/content/thucquan.pt": ['2_Viem_thuc_quan', '5_Ung_thu_thuc_quan'],
    "/content/daday.pt": ['3_Viem_da_day_HP_am', '4_Viem_da_day_HP_duong', '6_Ung_thu_da_day'],
    "/content/htt.pt": ['7_Loet_HTT']
}

# Thiết lập model_classes từ từ điển, nếu không khớp thì trả về ['polyp', 'esophagael cancer']
model_classes = model_classes_dict.get(model_weights, ['polyp', 'esophagael cancer'])

print("Input Video Name:", input_video_name)
print("Model Classes:", model_classes)

Input Video Name: CS101_TQ
Model Classes: ['2_Viem_thuc_quan', '5_Ung_thu_thuc_quan']


In [ ]:
class Colors:
    def __init__(self, num_colors=80):
        self.num_colors = num_colors
        self.color_palette = self.generate_color_palette()


    def generate_color_palette(self):
        hsv_palette = np.zeros((self.num_colors, 1, 3), dtype=np.uint8)
        hsv_palette[:, 0, 0] = np.linspace(0, 180, self.num_colors, endpoint=False)
        hsv_palette[:, :, 1:] = 255
        bgr_palette = cv2.cvtColor(hsv_palette, cv2.COLOR_HSV2BGR)
        return bgr_palette.reshape(-1, 3)

    def __call__(self, class_id):
        color = tuple(map(int, self.color_palette[class_id]))
        return color

In [ ]:
class ObjectDetection:
    def __init__(self, model_weights="yolov8s.pt", capture_index=0):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print("Using Device: ", self.device)
        self.model = self.load_model(model_weights)
        self.classes = self.model.names
        #self.classes[0] = 'gastritis'
        self.classes = model_classes
        self.colors = Colors(len(self.classes))
        self.font = cv2.FONT_HERSHEY_SIMPLEX
        self.capture_index = capture_index
        self.cap = self.load_capture()
        reid_weights = Path("osnet_x0_25_msmt17.pt")
        self.tracker = StrongSORT(reid_weights,
                                  torch.device(self.device),
                                  fp16 = False,
                                  )

    def load_model(self, weights):
        model = YOLO(weights)
        model.fuse()
        return model

    def predict(self, frame):
        results = self.model(frame, stream=True, verbose=False, conf=0.65, line_width=1)
        return results

    def draw_tracks(self, frame, tracks, txt_file, overlap_threshold=0.5):
        for track in tracks:
            x1, y1, x2, y2 = int(track[0]), int(track[1]), int(track[2]), int(track[3])
            id = int(track[4])
            conf = round(track[5], 2)
            class_id = int(track[6])
            class_name = self.classes[class_id]
            cv2.rectangle(frame, (x1,y1), (x2, y2), self.colors(class_id), 3)
            label = f'{class_name} {conf}' # hiển thị
            (w, h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 1.5, 3)
            cv2.rectangle(frame, (x1, y1-h-15), (x1+w, y1), self.colors(class_id), -1)
            cv2.putText(frame, label, (x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255) , 3)
            # Ghi kết quả vào file txt
            txt_file.write(f"{int(self.cap.get(cv2.CAP_PROP_POS_FRAMES))}, {id}, {x1}, {y1}, {x2-x1}, {y2-y1}, {conf}, -1, -1, -1\n")

        return frame


    def load_capture(self):
        cap = cv2.VideoCapture(self.capture_index)
        assert cap.isOpened()
        fps = cap.get(cv2.CAP_PROP_FPS)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
        video_name = "strongsort_" + input_video_name + ".mp4"
        self.writer = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
        return cap

    def write_seqinfo_ini(self, seq_name, seq_length, frame_rate, im_width, im_height, im_ext, im_dir):
        with open("seqinfo.ini", "w") as f:
            f.write("[Sequence]\n")
            f.write(f"name={seq_name}\n")
            f.write(f"imDir={im_dir}\n")  # Thay thế bằng thư mục chứa ảnh nếu cần
            f.write(f"frameRate={frame_rate}\n")
            f.write(f"seqLength={seq_length}\n")
            f.write(f"imWidth={im_width}\n")
            f.write(f"imHeight={im_height}\n")
            f.write(f"imExt={im_ext}\n")


    def __call__(self):
        tracker = self.tracker


        # Lấy thông tin từ video kết quả
        seq_name = "StrongSort"
        im_dir="img1"
        seq_length = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))
        frame_rate = self.cap.get(cv2.CAP_PROP_FPS)
        im_width = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        im_height = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        im_ext = ".jpg"  # Phần mở rộng của ảnh

        # Ghi thông tin vào file seqinfo.ini
        self.write_seqinfo_ini(seq_name, seq_length, frame_rate, im_width, im_height, im_ext, im_dir)

        # Mở file txt để ghi kết quả
        with open("results.txt", "w") as txt_file:
          while True:
              start_time = perf_counter()
              ret, frame = self.cap.read()
              if not ret:
                  break
              cv2.rectangle(frame, (0,30), (220,80), (255,255,255),-1 )
              detections = self.predict(frame)
              for dets in detections:
                  tracks = tracker.update(dets.boxes.data.to("cpu").numpy(), frame)
                  if len(tracks.shape) == 2 and tracks.shape[1] == 8:
                      frame = self.draw_tracks(frame, tracks, txt_file)
                  #print(tracks)
              end_time = perf_counter()
              fps = 1/np.round(end_time - start_time, 2)
              cv2.putText(frame, f'FPS: {int(fps)}', (20,70), self.font, 1.5, (0,255,0), 5)
              self.writer.write(frame)
              #cv2_imshow(frame)
              if cv2.waitKey(5) & 0xFF == 27:
                  break
          self.cap.release()
          self.writer.release()
          cv2.destroyAllWindows()

In [ ]:
detector = ObjectDetection(model_weights, test_vid)
detector()



Using Device:  cuda
Model summary (fused): 218 layers, 25840918 parameters, 0 gradients, 78.7 GFLOPs


2024-03-26 04:54:19.046 | SUCCESS  | boxmot.appearance.reid_model_factory:load_pretrained_weights:207 - Successfully loaded pretrained weights from "osnet_x0_25_msmt17.pt"
2024-03-26 04:54:19.048 | WARNING  | boxmot.appearance.reid_model_factory:load_pretrained_weights:211 - The following layers are discarded due to unmatched keys or layer size: ('classifier.weight', 'classifier.bias')
